## 3.4. 기계 번역: T5

기계 번역(Machine Translation)은 컴퓨터 프로그램을 활용해 한 언어의 텍스트를 다른 언어의 텍스트로 자동 변환하는 기술이다. 빠르고 저렴하지만 신조어, 관용구, 은유등 쫌 힘든 부분이 있다.

기계 번역 기술은 통계적 기계 번역(Statistical Machine Translation), 신경망 기계 번역(Neural Machine Translation) 등이 있다.
통계적 기계 번역은 원문과 번역문 쌍을 기반으로 단어 순서와 언어 패턴을 인식해 학습한다. 반면, 신경망 기계 번역은 심층 신경망 모델을 사용해 번역문과 단어 시퀀스 간의 관계를 학습한다.


### 3.4.1. T5

T5(Text-to-Text Transfer Transformer)는 구글에서 개발한 언어 모델로 인코더와 디코더로 이루어진 시퀀스-투-시퀀스(Seq2Seq) 모델이다. 이 모델은 모든 자연어 처리 과제를 텍스트-투-텍스트 형태의 데이터로 변환하고, 이를 시퀀스-투-시퀀스 문제로 인식해 해결한다. 입력과 출력이 모두 텍스트이다.

**기계 번역 과제**

* **입력 텍스트**: "오늘 날씨는 어때요?"
* **입력 시퀀스**: ["오늘, 날씨는, 어때요?"]
* **출력 시퀀스**: ["How's, the, weather, today?"]
* **출력 텍스트**: "How's the weather today?"



### 3.4.2. 기계 번역 모델 학습



In [1]:
from datasets import load_dataset

from transformers import T5TokenizerFast, T5ForConditionalGeneration

def preprocess_data(example, tokenizer):
    translation = example['translation']
    translation_source = ["en: " + instance["en"] for instance in translation]
    translation_target = ["ko: " + instance["ko"] for instance in translation]

    tokenized = tokenizer(
        translation_source,
        text_target=translation_target,
        truncation=True,
    )

    return tokenized


model_name = "KETI-AIR/long-ke-t5-small"
tokenizer = T5TokenizerFast.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

dataset = load_dataset("Helsinki-NLP/opus-100", "en-ko")

processed_dataset = dataset.map(
    lambda example: preprocess_data(example, tokenizer),
    batched=True,
    remove_columns=dataset["train"].column_names,
)

sample = processed_dataset["test"][0]
print(sample)
print("변환된 출발 언어:", tokenizer.decode(sample["input_ids"]))
print("변환된 도착 언어:", tokenizer.decode(sample["labels"]))

You are using a model of type longt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at KETI-AIR/long-ke-t5-small and are newly initialized: ['encoder.block.0.layer.0.SelfAttention.k.weight', 'encoder.block.0.layer.0.SelfAttention.o.weight', 'encoder.block.0.layer.0.SelfAttention.q.weight', 'encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.0.layer.0.SelfAttention.v.weight', 'encoder.block.1.layer.0.SelfAttention.k.weight', 'encoder.block.1.layer.0.SelfAttention.o.weight', 'encoder.block.1.layer.0.SelfAttention.q.weight', 'encoder.block.1.layer.0.SelfAttention.v.weight', 'encoder.block.2.layer.0.SelfAttention.k.weight', 'encoder.block.2.layer.0.SelfAttention.o.weight', 'encoder.block.2.layer.0.SelfAttention.q.weight', 'encoder.block.2.layer.0.SelfAttention.v.weight', 'encoder.block.3.layer.0.SelfAtt

{'input_ids': [20004, 20525, 20048, 20298, 20480, 20025, 20263, 20027, 20187, 20050, 43305, 20009, 21015, 20047, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [20004, 23477, 20048, 92, 14, 4256, 11, 1363, 71, 1133, 2951, 20371, 33, 16, 75, 242, 10, 513, 20047, 1]}
변환된 출발 언어: en: What makes you think I want an intro to anyone?</s>
변환된 도착 언어: ko: 내가 너를 누구에게 소개하고 싶어한다고 생각하니?</s>


OPUS-100 데이터세트는 다양한 언어 간 기계 번역 작업을 위해 구축된 대규모 병렬 말붕치 (Parallel corpus)다. 이중 en-ko는 영어와 한국어 간의 번역 쌍을 나타닌다. 이 데이터세트의 영어-한국어 부분은 100만개의 학습데이터와 각각 2,000개의 검증 및 테스트 데이터로 구성되어 있다.

데이터세트는`translation` 열로만 이루어져 있다. 이 열에는 영어 문장과 한국어 문장이 en과 ko 키를 갖는 딕셔너리 형태로 저장 되어 있다. 

`T5ForConditionalGeneration`을 사용할 때는 수행할 하위 작업을 프롬프트 형식으로 정의해야 한다. `en: 프롬프트`,를 이용해 모델에 전달한다. 모델은 이 프롬프트를 입력받아 처리한후 "ko: 프롬프트"를 출력한다.

`T5ForConditionalGeneration` 은 프롬프트 기반 접근법을 사용해 다양한 자연어 처리 작업을 수행할 수 있다. 번역 작업뿐만 아니라 요약, 질의응답 등의 장업도 프롬프트 형식을 적절히 정의하면 동일한 방식으로 수행할 수 있다.


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 첫 번째 GPU를 사용하도록 설정

from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5TokenizerFast, T5ForConditionalGeneration

def preprocess_data(example, tokenizer):
  translation = example['translation']  # 번역된 문장 쌍을 가져옴
  translation_source = ["en: " + instance["en"] for instance in translation]  # 영어 문장에 "en: " 접두사 추가
  translation_target = ["ko: " + instance["ko"] for instance in translation]  # 한국어 문장에 "ko: " 접두사 추가

  tokenized = tokenizer(
    translation_source,
    text_target=translation_target,
    truncation=True,  # 길이가 너무 길면 잘라냄
  )

  return tokenized  # 토큰화된 결과 반환

model_name = "KETI-AIR/long-ke-t5-small"  # 사용할 모델 이름
tokenizer = T5TokenizerFast.from_pretrained(model_name)  # 모델에 맞는 토크나이저 로드
model = T5ForConditionalGeneration.from_pretrained(model_name)  # 모델 로드

dataset = load_dataset("Helsinki-NLP/opus-100", "en-ko")  # 데이터셋 로드

processed_dataset = dataset.map(
  lambda example: preprocess_data(example, tokenizer),  # 데이터셋 전처리
  batched=True,  # 배치 단위로 처리
  remove_columns=dataset["train"].column_names,  # 원래 컬럼 제거
)


seq2seq2_collator = DataCollatorForSeq2Seq(
  tokenizer=tokenizer,
  padding="longest",  # 가장 긴 시퀀스에 맞춰 패딩
  return_tensors="pt",  # PyTorch 텐서로 반환
)

training_args = Seq2SeqTrainingArguments(
  output_dir="./runs/t5-translation",  # 출력 디렉토리
  logging_dir='./runs/t5-translation/logs',  # 로그 디렉토리
  per_device_train_batch_size=8,  # 훈련 배치 크기
  per_device_eval_batch_size=16,  # 평가 배치 크기
  learning_rate=2e-5,  # 학습률
  num_train_epochs=100,  # 학습 에포크 수
  eval_steps=200,  # 평가 스텝
  logging_steps=50,  # 로그 스텝
  save_steps=200,  # 저장 스텝
  seed=42,  # 랜덤 시드
)

trainer = Seq2SeqTrainer(
  model=model,  # 모델
  data_collator=seq2seq2_collator,  # 데이터 콜레이터
  args=training_args,  # 훈련 인자
  train_dataset=processed_dataset["train"].select(range(100000)),  # 훈련 데이터셋
  eval_dataset=processed_dataset["test"].select(range(1000)),  # 평가 데이터셋
)

# trainer.train()  # 모델 훈련

You are using a model of type longt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at KETI-AIR/long-ke-t5-small and are newly initialized: ['encoder.block.0.layer.0.SelfAttention.k.weight', 'encoder.block.0.layer.0.SelfAttention.o.weight', 'encoder.block.0.layer.0.SelfAttention.q.weight', 'encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.0.layer.0.SelfAttention.v.weight', 'encoder.block.1.layer.0.SelfAttention.k.weight', 'encoder.block.1.layer.0.SelfAttention.o.weight', 'encoder.block.1.layer.0.SelfAttention.q.weight', 'encoder.block.1.layer.0.SelfAttention.v.weight', 'encoder.block.2.layer.0.SelfAttention.k.weight', 'encoder.block.2.layer.0.SelfAttention.o.weight', 'encoder.block.2.layer.0.SelfAttention.q.weight', 'encoder.block.2.layer.0.SelfAttention.v.weight', 'encoder.block.3.layer.0.SelfAtt

T5ForConditionalGeneration 을 활용한 번역 모델 학습에서는 DataCollectorForSeq2Seq 클래스를 이용해 배치 단위로 입력 시퀀스와 출력 시퀀스에 대한 패딩 처리를 수행한다. T5와 BART등의 시퀀스-투-시퀀스 모델에서는 입력 데이터와 출력 데이터가 모두 텍스트 형태의 시퀀스로 구성되어 있다.
따라서 입력 ID와 레이블을 동일한 길이로 맞추기 위해 DataCollatorForSeq2Seq 클래스로 패딩 작업을 수행한다.

기계 번역은 출발어와 목적어 모두를 깊이 이해해야 하는 복잡한 자연어 처리 과제다. 그러므로 기존 엄청나게 오래 걸린다. tmux를 사용해야 한다.



In [3]:
import torch

model = T5ForConditionalGeneration.from_pretrained("./runs/t5-translation/best-model")

model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

data = "en: It's always great to acquire new knowledge."

inputs = tokenizer(data, return_tensors="pt").to(device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_length=512,
        num_beams=4,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))


ko: 새로운 지식을 얻을 때마다


In [11]:
# 기계 번역 모델 평가

import evaluate
from torch.utils.data import DataLoader

dataloader = DataLoader(
  processed_dataset["test"].select(range(100)),
  collate_fn=seq2seq2_collator,
  batch_size=4,
  shuffle=False
)

generated_translated = []

true_translated_ids = processed_dataset["test"].select(range(100))["labels"]

true_translated = tokenizer.batch_decode(true_translated_ids, skip_special_tokens=True)

with torch.no_grad():
    for batch in dataloader:
        batch = batch.to(device)
        outputs = model.generate(
            **batch,
            max_length=1026,
            num_beams=4,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        batch_translated = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        generated_translated.extend(batch_translated)

bleu = evaluate.load("bleu")

blue_score = bleu.compute(predictions=generated_translated, references=true_translated)
# 번역 평가 지표 로깅
print(blue_score)

import pandas as pd

pd.DataFrame({
    "source": [tokenizer.decode(instance["input_ids"], skip_special_tokens=True) for instance in processed_dataset["test"].select(range(100))],
    "reference": true_translated,
    "generated": generated_translated,
    "blue_score": blue_score["bleu"],
    "brevity_penalty": blue_score["brevity_penalty"],
    "length_ratio": blue_score["length_ratio"],
    "translation_length": blue_score["translation_length"],
    "reference_length": blue_score["reference_length"],
}).rename(columns={
    "source": "번역 대상 문장",
    "reference": "참조 번역",
    "generated": "생성된 번역",
    "blue_score": "BLEU 점수",
    "brevity_penalty": "단문 패널티",
    "length_ratio": "길이 비율",
    "translation_length": "번역 문장 길이",
    "reference_length": "참조 문장 길이",
})


{'bleu': 0.06456637647777932, 'precisions': [0.41389728096676737, 0.21530249110320285, 0.04112554112554113, 0.011049723756906077], 'brevity_penalty': 0.8093851751674768, 'length_ratio': 0.8254364089775561, 'translation_length': 662, 'reference_length': 802}


,번역 대상 문장,참조 번역,생성된 번역,BLEU 점수,단문 패널티,길이 비율,번역 문장 길이,참조 문장 길이
0,en: What makes you think I want an intro to an...,ko: 내가 너를 누구에게 소개하고 싶어한다고 생각하니?,ko: 내가 뭐라고 생각하는 거야?,0.064566,0.809385,0.825436,662,802
1,en: ( tires squealing ),ko: ( tires squealing ),ko: ( tires queing),0.064566,0.809385,0.825436,662,802
2,en: That would have put you in jail for a long...,ko: 그리고 당신의 마약 스캔들을 근절하려고 했고요 그건 당신을 오랫동안 감옥에 ...,ko: 그가 감옥에 갔을 때 당신은 그를 죽였을거야,0.064566,0.809385,0.825436,662,802
3,en: -He seems happy. -Happy is for ordinary pe...,ko: 성형외과에나 기웃거리게 만들고 말이지,ko: - 행복한 사람들이야,0.064566,0.809385,0.825436,662,802
4,en: I'll find out where she's from and get the...,ko: 그녀가 어디소속이지 찾아서 물러나도록 할거야,ko: 그들을 찾아낼 수 있는 곳이 어디야?,0.064566,0.809385,0.825436,662,802
...,...,...,...,...,...,...,...,...
95,"en: I do two weeks on, two weeks off.",ko: 2주 일하고 2주 쉬죠,ko: 2주 정도 기다려,0.064566,0.809385,0.825436,662,802
96,en: The way you left me.,ko: 그래서 떠난거군,ko: 당신은 나를 떠났어요,0.064566,0.809385,0.825436,662,802
97,en: We had to restrain him. I requested a psyc...,ko: 저희가 묶어뒀습니다 정신과 진찰도 요청했습니다,ko: 우리는 그를 보호하기로 했어요,0.064566,0.809385,0.825436,662,802
98,"en: And yours, I'm afraid. Idea of the month g...",ko: 당신도 물론이고,ko: 그리고 내가 당신의 계획을 두려워하는 거야,0.064566,0.809385,0.825436,662,802


MLFlow 설정을 잘못해서 정확히 얼마나 걸렸는지는 모르겠다. 로그도 지워서 더 짜증난다. (ㅋㅋㅋ) 그래도 일단 눈뜨기 전에는 학습이 끝났다. 

```python
per_device_train_batch_size=16,  # 훈련 배치 크기
per_device_eval_batch_size=24,  # 평가 배치 크기
learning_rate=2e-5,  # 학습률
num_train_epochs=20,  # 학습 에포크 수
eval_steps=250,  # 평가 스텝
logging_steps=250,  # 로그 스텝
save_steps=250,  # 저장 스텝
seed=42,  # 랜덤 시드
fp16=True,  # 플로트 16 비트 연산 활성화
gradient_accumulation_steps=2,  # 그래디언트 축적 단계를 4로 설정 (메모리 사용량 줄임)
dataloader_num_workers=4,  # 데이터 로딩에 사용할 서브프로세스 수 (병렬화)
dataloader_pin_memory=True,  # 텐서를 CUDA 고정 메모리에 할당하여 데이터 로딩 속도 향상
```
그나마 최적이라고 생각되는 하이퍼파라미터를 적용했다. 안타깝게도 뭘 잘못했는지는 모르겠지만 GPU를 늘리면 오히려 2배의 예상 시간이 걸린다. 그래서 1개로 돌렸다. (?)
그래서 책에는 없는 내용을 용케 찾아서 적용했다. 번역 결과는 참으로 참담하기 그지없다. 그래도 이정도 결과가 나오는 것도 전기값은 하는 것 같다.

